In [ ]:
# CHANGE THIS if your dataset folder name in /kaggle/input is different
DATASET_DIR = "/kaggle/input/indoor-fire-v1-rgb1-darknet"

# Working paths (don’t change)
KAGGLE_WORKING = "/kaggle/working"
EXTRACT_DIR    = f"{KAGGLE_WORKING}/extracted"
BACKUP_DIR     = f"{KAGGLE_WORKING}/backup"

import os, shutil, glob, random, re, time, subprocess, sys
os.makedirs(EXTRACT_DIR, exist_ok=True)
os.makedirs(BACKUP_DIR, exist_ok=True)
print("DATASET_DIR:", DATASET_DIR)

In [ ]:
import os, sys, subprocess, textwrap

print("== nvidia-smi ==")
os.system("nvidia-smi || echo 'nvidia-smi not available'")

print("\n== CUDA toolchain ==")
os.system("nvcc --version || echo 'nvcc not available (OK if Darknet uses preinstalled CUDA)'")

# Make sure CUDA libs are visible (belt-and-suspenders)
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda/lib64:" + os.environ.get("LD_LIBRARY_PATH","")

# Hard fail if GPU isn’t present
smicode = os.system("nvidia-smi > /dev/null 2>&1")
if smicode != 0:
    raise SystemExit("🚫 No GPU detected by nvidia-smi. Check Notebook -> Settings -> Accelerator -> GPU (T4).")
print("✅ GPU detected")

In [ ]:
import os, shutil, subprocess, sys

# make sure we are in a valid directory
os.chdir("/kaggle/working")

# remove old clone if it exists
shutil.rmtree("darknet", ignore_errors=True)

print("== Cloning AlexeyAB/darknet ==")
res = subprocess.run(
    ["git","clone","--depth","1","https://github.com/AlexeyAB/darknet.git","darknet"],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
)
print(res.stdout[-400:])
if res.returncode != 0:
    print(res.stderr[-400:])
    raise SystemExit("Git clone failed")

print("✅ Clone complete, darknet directory ready at /kaggle/working/darknet")

In [ ]:
import os, re, shutil, subprocess, textwrap, sys

MK = "/kaggle/working/darknet/Makefile"
assert os.path.exists(MK), "Makefile not found; clone step must succeed first."

txt = open(MK,"r").read()

# Ensure GPU/CUDNN/CUDNN_HALF are ON (leave OPENCV as-is for now)
txt = re.sub(r"^GPU=0",        "GPU=1",        txt, flags=re.M)
txt = re.sub(r"^CUDNN=0",      "CUDNN=1",      txt, flags=re.M)
txt = re.sub(r"^CUDNN_HALF=0", "CUDNN_HALF=1", txt, flags=re.M)

# If you hit OpenCV pkg-config errors later, flip OPENCV to 0.
# For now we keep it ON since your log shows it worked.
# txt = re.sub(r"^OPENCV=1", "OPENCV=0", txt, flags=re.M)

# Remove explicit -lcuda (not needed on Kaggle and causes link error)
# Do it both in lines with LIBS and generic occurrences.
txt = re.sub(r"\s-lcuda(\s|$)", r" ", txt)

# Make sure arch includes sm_75 (T4). Keep existing others.
if re.search(r"^ARCH=", txt, flags=re.M):
    # append compute_75 if not present
    if "sm_75" not in txt:
        txt = re.sub(r"^ARCH=.*", 
                     lambda m: m.group(0) + " -gencode arch=compute_75,code=[sm_75,compute_75]", 
                     txt, flags=re.M)
else:
    # Fallback: define ARCH line
    txt += "\nARCH= -gencode arch=compute_75,code=[sm_75,compute_75]\n"

open(MK,"w").write(txt)
print("✅ Makefile patched: GPU/CUDNN on, -lcuda removed, sm_75 ensured.")


In [ ]:
import os, shutil, glob

# Define your paths first
DATASET_DIR  = "/kaggle/input/indoor-fire-v1-rgb1-darknet"  # <--- adjust if your dataset folder name changes
EXTRACT_DIR  = "/kaggle/working/extracted"
os.makedirs(EXTRACT_DIR, exist_ok=True)

print("== Copy dataset into working directory ==")
if not os.path.exists(DATASET_DIR):
    os.system("ls -la /kaggle/input/")
    raise SystemExit(f"❌ Dataset folder not found: {DATASET_DIR}")

# Shallow copy (keeps structure)
for item in os.listdir(DATASET_DIR):
    s = os.path.join(DATASET_DIR, item)
    d = os.path.join(EXTRACT_DIR, item)
    if os.path.isdir(s):
        if not os.path.exists(d):
            shutil.copytree(s, d)
    else:
        shutil.copy2(s, d)

# Find images that have corresponding YOLO txt labels
img_exts = (".jpg",".jpeg",".png",".bmp",".JPG",".PNG",".JPEG",".BMP")
imgs = []
for ext in img_exts:
    for p in glob.glob(os.path.join(EXTRACT_DIR,"**",f"*{ext}"), recursive=True):
        if os.path.exists(os.path.splitext(p)[0] + ".txt"):
            imgs.append(os.path.abspath(p))

print(f"✅ Found labeled images: {len(imgs)}")
if len(imgs) < 10:
    # show a quick peek to debug structure
    os.system(f"find {EXTRACT_DIR} -maxdepth 2 -type f | head -n 30")
    raise SystemExit("❌ Too few labeled images. Check that .txt label files sit beside images (same basename).")


In [ ]:
import os, random  # <-- add random here

# Shuffle and split dataset
random.shuffle(imgs)
split = int(0.8 * len(imgs))
train_list = imgs[:split]
valid_list = imgs[split:]

# Create Darknet data folder
os.makedirs("/kaggle/working/darknet/data", exist_ok=True)

# Write train/valid splits
with open("/kaggle/working/darknet/data/train.txt","w") as f:
    f.write("\n".join(train_list))
with open("/kaggle/working/darknet/data/valid.txt","w") as f:
    f.write("\n".join(valid_list))

# Write class names
num_classes = 4
with open("/kaggle/working/darknet/data/obj.names","w") as f:
    f.write("fire\nno_fire\nobjects\nhuman\n")

# Write data config
BACKUP_DIR = "/kaggle/working/backup"  # define backup dir if not yet
os.makedirs(BACKUP_DIR, exist_ok=True)

with open("/kaggle/working/darknet/data/obj.data","w") as f:
    f.write(
        f"classes = {num_classes}\n"
        f"train = data/train.txt\n"
        f"valid = data/valid.txt\n"
        f"names = data/obj.names\n"
        f"backup = {BACKUP_DIR}\n"
    )

print(f"✅ train: {len(train_list)}, valid: {len(valid_list)}")

# Quick path check
with open('/kaggle/working/darknet/data/train.txt') as f:
    sample = [l.strip() for l in f.readlines()[:5] if l.strip()]
print("Sample paths:", sample)
for sp in sample:
    assert os.path.exists(sp), f"Missing sample path: {sp}"

In [ ]:
import shutil, glob, re, os

# If num_classes isn't defined earlier, set it here:
try:
    num_classes
except NameError:
    num_classes = 4  # adjust if needed

cfg_dir = "/kaggle/working/darknet/cfg"
candidates = ["yolov4-tiny-custom.cfg","yolov4-tiny-3l.cfg","yolov4-tiny.cfg"]
src_cfg = next((os.path.join(cfg_dir,c) for c in candidates if os.path.exists(os.path.join(cfg_dir,c))), None)
if src_cfg is None:
    ls = glob.glob(os.path.join(cfg_dir,"*yolov4*tiny*.cfg"))
    if not ls:
        os.system(f"ls -la {cfg_dir}")
        raise SystemExit("❌ No yolov4-tiny cfg file found")
    src_cfg = ls[0]

dst_cfg = os.path.join(cfg_dir,"yolov4-tiny-obj.cfg")
shutil.copy2(src_cfg, dst_cfg)

# ----- load cfg text -----
with open(dst_cfg, "r", errors="ignore") as f:
    text = f.read()

# ----- robust key=value replacements (tolerant to spaces/casing) -----
def sub_kv(t, key, value):
    # replace lines like "key = something" or "key=something"
    pattern = re.compile(rf"(?im)^\s*{re.escape(key)}\s*=\s*.*$")
    if pattern.search(t):
        return pattern.sub(f"{key}={value}", t)
    else:
        # if the key doesn't exist, insert near top (after first line)
        lines = t.splitlines(True)
        insert_at = min(10, len(lines))
        lines.insert(insert_at, f"{key}={value}\n")
        return "".join(lines)

# core training knobs
text = sub_kv(text, "batch", "64")
text = sub_kv(text, "subdivisions", "32")
text = sub_kv(text, "width", "416")
text = sub_kv(text, "height", "416")

# schedule: force 8000 and 6400,7200
text = sub_kv(text, "max_batches", "8000")
text = sub_kv(text, "steps", "6400,7200")

# optional learning rate if missing
if re.search(r"(?im)^\s*learning_rate\s*=", text) is None:
    lines = text.splitlines(True)
    lines.insert(10, "learning_rate=0.001\n")
    text = "".join(lines)

# ----- write back (so we can parse yolo blocks next) -----
with open(dst_cfg, "w") as f:
    f.write(text)

# ----- update classes and filters in [yolo] blocks -----
with open(dst_cfg, "r") as f:
    lines = f.readlines()

filters = (num_classes + 5) * 3  # e.g., (4+5)*3 = 27
yolo_idxs = [i for i,l in enumerate(lines) if l.strip().lower() == "[yolo]"]

for yi in yolo_idxs:
    # set classes in the [yolo] block
    j = yi + 1
    while j < len(lines) and "[" not in lines[j]:
        if lines[j].strip().lower().startswith("classes="):
            lines[j] = f"classes={num_classes}\n"; break
        j += 1
    # set filters in the preceding [convolutional] block
    k = yi - 1
    while k >= 0 and lines[k].strip().lower() != "[convolutional]":
        k -= 1
    if k >= 0:
        m = k + 1
        while m < yi:
            if lines[m].strip().lower().startswith("filters="):
                lines[m] = f"filters={filters}\n"; break
            m += 1

with open(dst_cfg, "w") as f:
    f.writelines(lines)

print("✅ cfg ready:", dst_cfg)

# quick verify (print the lines we care about)
with open(dst_cfg) as f:
    cfg_txt = f.read()
for key in ["batch","subdivisions","width","height","max_batches","steps"]:
    m = re.search(rf"(?im)^\s*{key}\s*=\s*.*$", cfg_txt)
    if m: print(m.group(0))
print("classes/filters updated for all [yolo] heads.")

# download pretrain if needed
if not os.path.exists("/kaggle/working/yolov4-tiny.conv.29"):
    rc = os.system(
        "wget -q https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29 "
        "-O /kaggle/working/yolov4-tiny.conv.29"
    )
    if rc != 0:
        raise SystemExit("❌ Failed to download yolov4-tiny.conv.29")
print("✅ pretrain present")


In [ ]:
import os, re

# stay in your same path
os.chdir("/kaggle/working/darknet")

print("== Probe for driver libs ==")
os.system("ls -l /usr/local/cuda/compat/libcuda* 2>/dev/null || true")
os.system("ls -l /usr/local/cuda/lib64/stubs/libcuda* 2>/dev/null || true")

MK = "Makefile"
txt = open(MK, "r").read()

# Ensure GPU/CUDNN/HALF ON; keep OpenCV OFF
txt = re.sub(r"^GPU=0\b",        "GPU=1",        txt, flags=re.M)
txt = re.sub(r"^CUDNN=0\b",      "CUDNN=1",      txt, flags=re.M)
txt = re.sub(r"^CUDNN_HALF=0\b", "CUDNN_HALF=1", txt, flags=re.M)
txt = re.sub(r"^OPENCV=1\b",     "OPENCV=0",     txt, flags=re.M)

# Ensure sm_75 (T4) arch is included
if re.search(r"^ARCH\s*=", txt, flags=re.M):
    if "sm_75" not in txt:
        txt = re.sub(r"^ARCH\s*=.*",
                     lambda m: m.group(0) + " -gencode arch=compute_75,code=[sm_75,compute_75]",
                     txt, flags=re.M)
else:
    txt += "\nARCH= -gencode arch=compute_75,code=[sm_75,compute_75]\n"

# Clean old link hacks so we can re-append cleanly
txt = re.sub(r".*(kaggle driver link|kaggle link fix|driver link).*?\n", "", txt, flags=re.I)
txt = re.sub(r".*(LIBS|LDFLAGS)\s*\+=.*-lcuda.*\n", "", txt)
txt = re.sub(r".*(LIBS|LDFLAGS)\s*\+=.*(-lcudart|-lcublas|-lcurand|-lcudnn).*?\n", "", txt)
txt = re.sub(r".*libcuda\.so\.1.*\n", "", txt)
txt = re.sub(r".*/usr/lib/x86_64-linux-gnu.*\n", "", txt)
open(MK, "w").write(txt)

# Decide which driver path to rpath
have_compat = os.path.exists("/usr/local/cuda/compat/libcuda.so") or os.path.exists("/usr/local/cuda/compat/libcuda.so.1")
have_stubs  = os.path.exists("/usr/local/cuda/lib64/stubs/libcuda.so")

rpath_line = ""
if have_compat:
    rpath_line = "-Wl,-rpath,/usr/local/cuda/compat -L/usr/local/cuda/compat"
elif have_stubs:
    rpath_line = "-Wl,-rpath,/usr/local/cuda/lib64/stubs -L/usr/local/cuda/lib64/stubs"

# Force CUDA libs at the very end, grouped, and disable as-needed so nothing gets dropped
fix = [
    "\n# --- Kaggle driver/runtime link fix ---\n",
    "LDFLAGS+= -L/usr/local/cuda/lib64 -L/usr/local/cudnn/lib64 {rpath} ".format(rpath=rpath_line),
    "         -Wl,--no-as-needed -Wl,--start-group ",
    "         -lcudart -lcublas -lcurand -lcudnn -lcuda ",
    "         -Wl,--end-group\n",
]

with open(MK, "a") as f:
    f.write("".join(fix))

print("✅ Makefile patched (CUDA libs forced at end, grouped). Rebuilding...")

# Rebuild
os.system("make clean > /dev/null 2>&1")
rc = os.system("make -j2 > build.log 2>&1")
print("make rc=", rc)
if rc != 0:
    print("❌ Build failed — last 200 lines:")
    os.system("tail -n 200 build.log || cat build.log")
    raise SystemExit("Darknet GPU build failed. See log above.")

# Verify binary & linkage
assert os.path.exists("./darknet"), "darknet binary missing after build"
os.system("chmod +x ./darknet")
print("✅ darknet binary ready")

print("\n== Verify libcuda & cudart mapping ==")
os.system("ldd ./darknet | egrep 'libcuda|libcudart|cublas|cudnn|curand' || true")

print("\n== Confirm link line had our group ==")
os.system("tail -n 200 build.log | egrep -i 'start-group|lcudart|lcublas|lcurand|lcuda|lcudnn|rpath' || true")


In [ ]:
import os, sys, time, re, datetime, statistics, subprocess, shutil, tempfile, json
from pathlib import Path

# ===================== USER TOGGLE =====================
EMERGENCY_ONLY = False   # <- set True to skip training and ONLY export the newest checkpoint
# =======================================================

os.chdir("/kaggle/working/darknet")

# --- paths (unchanged) ---
CFG_PATH   = "cfg/yolov4-tiny-obj.cfg"
DATA_PATH  = "data/obj.data"
NAMES_PATH = "data/obj.names"
BACKUP_DIR = "/kaggle/working/backup"     # Darknet default backup dir
EXPORT_DIR = "/kaggle/working"             # where we drop last.weights + ZIP

# --- read max_batches from cfg for ETA ---
max_batches = 8000
try:
    with open(CFG_PATH) as f:
        for ln in f:
            m = re.search(r"^\s*max_batches\s*=\s*(\d+)", ln)
            if m:
                max_batches = int(m.group(1)); break
except Exception:
    pass

# --- helper: find latest .weights in backup ---
def find_latest_weights(bk_dir=BACKUP_DIR):
    if not os.path.isdir(bk_dir):
        return None
    ws = [str(p) for p in Path(bk_dir).glob("*.weights")]
    if not ws: return None
    # prefer *_last.weights; else newest by mtime
    last = [p for p in ws if p.endswith("_last.weights")]
    if last:
        return max(last, key=os.path.getmtime)
    return max(ws, key=os.path.getmtime)

# --- helper: export newest checkpoint for download/resume ---
def export_latest_checkpoint(exp_dir=EXPORT_DIR, bk_dir=BACKUP_DIR, extra_note=None):
    w = find_latest_weights(bk_dir)
    if not w:
        print("🚫 No .weights found to export in", bk_dir)
        return (None, None)

    # Mirror to /kaggle/working/last.weights for easy resume in next runs
    last_copy = os.path.join(exp_dir, "last.weights")
    try:
        shutil.copy2(w, last_copy)
    except Exception as e:
        print("⚠️ Could not copy to", last_copy, "->", e)

    # Prepare a small bundle with config + data for clean resume elsewhere
    iter_tag = ""
    m = re.search(r"(\d+)\.weights$", w)
    if m: iter_tag = f"_{m.group(1)}"
    bundle_name = f"resume_yolov4tiny{iter_tag}"  # no extension; make_archive adds .zip
    bundle_root = tempfile.mkdtemp(prefix="resume_bundle_")
    try:
        files_to_pack = []

        # Required files if present
        for pth in [w, CFG_PATH, DATA_PATH, NAMES_PATH, "training.log"]:
            if os.path.exists(pth):
                dst = os.path.join(bundle_root, os.path.basename(pth))
                try:
                    shutil.copy2(pth, dst)
                    files_to_pack.append(dst)
                except Exception as e:
                    print(f"⚠️ Skipped {pth}: {e}")

        # Add a tiny README with resume steps
        readme = os.path.join(bundle_root, "HOW_TO_RESUME.txt")
        with open(readme, "w") as f:
            f.write(
f"""HOW TO RESUME (Darknet):
1) Put the .weights file into a 'backup' folder next to your cfg/data files.
   Example structure:
     /.../darknet/
       ├── cfg/yolov4-tiny-obj.cfg
       ├── data/obj.data
       ├── data/obj.names
       ├── backup/{os.path.basename(w)}
2) Launch training with your usual command. Darknet will auto-pick the latest weights
   if your script scans the backup folder; otherwise pass the weight path explicitly:
   ./darknet detector train data/obj.data {CFG_PATH} backup/{os.path.basename(w)} -gpus 0

Notes:
- A copy of your latest weights is also placed at: {last_copy}
- If your script already scans {BACKUP_DIR}, just keep the file there and rerun.
"""
            )
        files_to_pack.append(readme)

        # Optional metadata
        meta = {
            "export_time": datetime.datetime.now().isoformat(),
            "source_backup_dir": bk_dir,
            "selected_weights": w,
            "max_batches": max_batches,
            "note": extra_note or ""
        }
        meta_path = os.path.join(bundle_root, "resume_meta.json")
        with open(meta_path, "w") as f:
            json.dump(meta, f, indent=2)
        files_to_pack.append(meta_path)

        # Make ZIP in EXPORT_DIR
        zip_base = os.path.join(exp_dir, bundle_name)
        zip_path = shutil.make_archive(zip_base, "zip", bundle_root)
        print(f"✅ Exported latest checkpoint:\n   Weights copy : {last_copy}\n   Bundle (ZIP) : {zip_path}")
        return (last_copy, zip_path)
    finally:
        try:
            shutil.rmtree(bundle_root, ignore_errors=True)
        except: pass

# --- emergency-only mode: just export & exit ---
if EMERGENCY_ONLY:
    export_latest_checkpoint(extra_note="Manual emergency export (EMERGENCY_ONLY=True).")
    raise SystemExit(0)

# --- sanity: built with GPU? ---
out = subprocess.run(["./darknet","detector","train","-h"],
                     stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True).stdout
print(out.splitlines()[0] if out else "")
if "GPU isn't used" in out:
    raise SystemExit("❌ Darknet binary is CPU-only. Rebuild with GPU flags before training.")

# --- config: turn mAP on/off ---
USE_MAP = False   # <- set True when you want periodic mAP

# --- choose weights (resume if available) ---
weights_file = "/kaggle/working/yolov4-tiny.conv.29"
if os.path.isdir(BACKUP_DIR):
    import glob
    w = sorted(glob.glob(os.path.join(BACKUP_DIR,"*.weights")), key=os.path.getmtime)
    if w:
        weights_file = w[-1]  # resume from latest checkpoint

# --- training command (streamed) ---
cmd = ["./darknet","detector","train", DATA_PATH, CFG_PATH, weights_file, "-gpus","0,1"]
if USE_MAP:
    cmd.insert(6, "-map")  # add mAP right before -gpus

print("== Training cmd ==\n", " ".join(cmd))

# --- regex to parse progress lines & weight saves ---
iter_re = re.compile(r"^\s*(\d+):\s*([\d.]+),\s*([\d.]+)\s*avg loss.*?([0-9.]+)\s*seconds", re.I)
save_re = re.compile(r"Saving weights to\s+(.+?\.weights)\s*$")

start = time.time()
last_status_print = 0.0
secs_hist = []
last_map_line = None
last_saved_weights = None

with open("training.log", "a", buffering=1) as logf:
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                            text=True, bufsize=1, universal_newlines=True)
    try:
        for line in proc.stdout:
            logf.write(line)
            s = line.rstrip()

            # progress
            m = iter_re.search(s)
            if m:
                cur_iter = int(m.group(1))
                avg_loss = float(m.group(3))
                sec_it   = float(m.group(4))
                secs_hist.append(sec_it)
                if len(secs_hist) > 50: secs_hist = secs_hist[-50:]
                sec_per_iter = statistics.mean(secs_hist)
                eta_secs = max(0, int((max_batches - cur_iter) * sec_per_iter))
                pct = 100.0 * cur_iter / max_batches
                bar_len, filled = 28, int(28 * pct / 100)
                bar = "█"*filled + "-"*(bar_len-filled)
                now = time.time()
                if now - last_status_print > 3:
                    print(f"[{cur_iter:>6}/{max_batches:<6}] [{bar}] {pct:5.1f}% | avg loss {avg_loss:.4f} | {sec_per_iter:.2f}s/it | ETA {datetime.timedelta(seconds=eta_secs)}")
                    last_status_print = now

            # mAP lines
            if USE_MAP and ("mAP@" in s):
                last_map_line = s
                print("📈", s)

            # weight saving events -> immediately mirror + mini-export
            msave = save_re.search(s)
            if msave:
                saved_path = msave.group(1).strip()
                last_saved_weights = saved_path
                print("💾", s)

                # Mirror the just-saved weights to /kaggle/working/last.weights
                try:
                    shutil.copy2(saved_path, os.path.join(EXPORT_DIR, "last.weights"))
                except Exception as e:
                    print("⚠️ Could not mirror to last.weights:", e)

                # Lightweight export: update a tiny JSON pointer (handy after crashes)
                try:
                    with open(os.path.join(EXPORT_DIR, "last_checkpoint.json"), "w") as fp:
                        json.dump({"last_saved_weights": saved_path,
                                   "time": datetime.datetime.now().isoformat()}, fp)
                except Exception as e:
                    print("⚠️ Could not write last_checkpoint.json:", e)

            if ("nan" in s.lower()) or ("CUDA Error" in s) or ("No such file" in s):
                print("⚠️", s)

        proc.wait()
    finally:
        try: proc.terminate()
        except: pass

rc = proc.returncode
print(f"\n== Training finished. Exit code: {rc}, elapsed: {(time.time()-start)/60:.1f} min ==")

# --- always do a final export so you can download even if training stopped ---
_ = export_latest_checkpoint(extra_note=f"Final export after training exit (rc={rc}).")

if rc != 0:
    print("❌ Training failed — tail of training.log:")
    os.system("tail -n 150 training.log || true")
else:
    print("\n== Recent 'Saving weights to' ==")
    os.system("grep -n 'Saving weights to' training.log | tail -n 5 || true")
    if USE_MAP and last_map_line:
        print("Last mAP line:", last_map_line)

print("\n👉 To do a manual emergency export without training, set EMERGENCY_ONLY=True and rerun this cell.")
print("📦 Download paths you’ll see in the file browser:\n  - /kaggle/working/last.weights\n  - /kaggle/working/resume_yolov4tiny_*.zip")


In [ ]:
#BELOW ARE FOR TESTING ACCURACY AND PRECISION

In [ ]:
#mAP on your validation set (per-class AP + overall mAP)
cd /kaggle/working/darknet

# pick the weights you want to evaluate
WEIGHTS="/kaggle/working/last.weights"      # or backup/yolov4-tiny-obj_best.weights if you have it

# mAP @ IoU=0.50 (PASCAL VOC style) with dense PR curve (101 points)
./darknet detector map data/obj.data cfg/yolov4-tiny-obj.cfg "$WEIGHTS" \
  -iou_thresh 0.50 -thresh 0.25 -points 101 -dont_show > map.log 2>&1

# show the key lines
tail -n 60 map.log


In [ ]:
#Quick recall/precision snapshot (fast sanity check)
cd /kaggle/working/darknet
./darknet detector recall data/obj.data cfg/yolov4-tiny-obj.cfg "$WEIGHTS" \
  -thresh 0.25 -iou_thresh 0.50 > recall.log 2>&1
tail -n 40 recall.log